In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df1 = pd.read_csv("ratings.csv")
df2 = pd.read_csv("good_reads.csv")
df1.head(15)
df2.head(15)

,user_id,book_id,rating,timestamp
0,900,A,4,08/08/18
1,901,A,4,08/09/18
2,902,B,4,08/10/18
3,900,B,3,08/11/18
4,904,C,4,08/12/18
5,905,C,5,08/13/18
6,900,D,5,08/14/18
7,902,D,2,08/15/18
8,901,E,3,08/16/18
9,902,E,4,08/17/18


In [25]:
#Convierto a formato de fecha x las dudas

df1['timestamp'] = pd.to_datetime(df1['timestamp'])
df1.head()

,user_id,book_id,rating,timestamp
0,900,A,4,2018-08-08
1,901,A,4,2018-08-09
2,902,B,4,2018-08-10
3,900,B,3,2018-08-11
4,904,C,4,2018-08-12


In [43]:
#Obtener un DataFrame que indique el TOP5 de Ebooks en el sitio de Ebooks. (Para
#este punto se puede ignorar el segundo DataFrame).

top_ratings = df1.groupby('book_id')['rating'].agg(np.mean).reset_index()
top_ratings.sort_values('rating', ascending = False, inplace = True)
top_ratings.head(15)

,book_id,rating
6,G,4.75
2,C,4.50
3,D,4.25
5,F,4.00
0,A,3.50
1,B,3.50
4,E,3.50


In [42]:
#Ahora me voy a buscar los nombres de los libros al otro DF 
names_for_id = df2[['book_id','book_name']]
top_books = pd.merge(top_ratings, names_for_id, on = 'book_id', how = 'inner')
top_books.head(15)

,book_id,rating,book_name
0,G,4.75,“good grades”
1,C,4.50,“on Spotify”
2,D,4.25,“from the album”
3,F,4.00,“for clear skin”
4,A,3.50,“Stream Karma”
5,B,3.50,“by MARINA”
6,E,3.50,“LOVE + FEAR”


In [26]:
#Obtener un DataFrame que indique qué libros tienen una diferencia de rating promedio
#mayor al 20% entre el sitio de Ebooks y GoodReads.

#Ahora hago un join que incluya el avg_rate del sitio de good_reads
#Vuelvo a trabajar con un df que agrupo por book_id y sacó el promedio

mean_good_reads = df1.groupby('book_id')['rating'].agg(np.mean).reset_index()
mean_good_reads.head()

,book_id,rating
0,A,3.50
1,B,3.50
2,C,4.50
3,D,4.25
4,E,3.50
5,F,4.00
6,G,4.75


In [39]:
#Joineo con el otro sitio (que ya tiene el promedo hecho)

#Join
df4 = pd.merge(mean_good_reads, df2, on= 'book_id', how = 'inner')

#Creo columna de diferencia potencial
df4['dev_rating'] = np.abs(((df4['rating'] - df4['avg_rating']) / df4['rating'])*100)

df4.head(10)

,book_id,rating,book_name,avg_rating,dev_rating
0,A,3.50,“Stream Karma”,4.2,20.000000
1,B,3.50,“by MARINA”,4.1,17.142857
2,C,4.50,“on Spotify”,3.2,28.888889
3,D,4.25,“from the album”,4.2,1.176471
4,E,3.50,“LOVE + FEAR”,5.0,42.857143
5,F,4.00,“for clear skin”,3.2,20.000000
6,G,4.75,“good grades”,4.3,9.473684


In [41]:
df5 = df4.loc[df4.dev_rating >= 20]
df5.head(5)

,book_id,rating,book_name,avg_rating,dev_rating
0,A,3.5,“Stream Karma”,4.2,20.000000
2,C,4.5,“on Spotify”,3.2,28.888889
4,E,3.5,“LOVE + FEAR”,5.0,42.857143
